In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2

In [ ]:
label_data = {
    'positive':1,
    'negative':0
}

In [ ]:
X_train = []
y_train = []

path_pos = 'datasets/datasets/train/positive/'

for i in os.listdir(path_pos):
    try:
        image = cv2.imread(os.path.join(path_pos, i))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (384, 384))
        image = image.reshape(384, 384, 1)
        image = image / 255.0
        X_train.append(image)
        y_train.append(1)
        
    except:
        continue

path_neg = 'datasets/datasets/train/negative/'

for i in os.listdir(path_neg):
    try:
        image = cv2.imread(os.path.join(path_neg, i))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (384, 384))
        image = image.reshape(384, 384, 1)
        image = image / 255.0
        X_train.append(image)
        y_train.append(0)
        
    except:
        continue
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape
y_train.shape

In [ ]:
X_test = []

for i in os.listdir('DLAI3_CXR_Validation_Set/'):

    image_path = os.path.join('DLAI3_CXR_Validation_Set/', i) 
    
    try:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (384, 384))
        image = image.reshape(384, 384, 1)
        image = image / 255.0 
        
        X_test.append(image)   
    except:
        continue

X_test = np.array(X_test)

X_test.shape

In [ ]:
len(os.listdir('DLAI3_CXR_Validation_Set/'))

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train, num_classes=2)

In [ ]:
y_train

In [ ]:
from sklearn.model_selection import train_test_split

X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size=0.20, 
    random_state=42,
)

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu', input_shape=(384, 384, 1))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))



model.add(Dense(units=2, activation='softmax')) 


model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train_final, y_train_final, validation_data=(X_val, y_val),epochs=10)

In [ ]:
df_sub = pd.read_csv('sampleSubmission.csv')

In [ ]:
df_sub

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred = [np.argmax(i) for i in y_pred]

In [ ]:
y_pred

In [ ]:
df_sub['label'] = y_pred

In [ ]:
df_sub

In [ ]:
df_sub.to_csv('sub.csv', index=False, index_label=False)